Preliminary work:
    - count 0s, 0.1s, and missing values

In [125]:
import synapseclient
import pandas as pd
import numpy as np
import json

syn = synapseclient.Synapse()
syn.login(silent=True)

overall_scores = 'syn25575156'
overall_scores = syn.tableQuery(f"select * from {overall_scores}").asDataFrame()
# overall_scores = pd.read_json(overall_scores.path, orient='records')

overall_scores.columns

Index(['GeneName', 'ENSG', 'Overall', 'Overall_rank', 'GeneticsScore',
       'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore',
       'SM_Druggability_bucket', 'safety_bucket', 'feasibility_bucket',
       'ABability_bucket', 'new_modality_bucket', 'tissue_engagement_bucket',
       'Pharos_class', 'classification', 'safety_bucket_definition',
       'feasibility_bucket_definition', 'ABability_bucket_definition',
       'new_modality_bucket_definition', 'tissue_engagement_bucket_definition',
       'isScored_genetics', 'isScored_omics', 'isScored_lit',
       'isScored_neuropath'],
      dtype='object')

In [126]:
interesting_columns = ['ENSG', 'Overall', 'GeneticsScore', 'OmicsScore', 'LiteratureScore', 'FlyNeuroPathScore']
scored = ['isScored_genetics', 'isScored_omics', 'isScored_lit', 'isScored_neuropath']
mapping = dict(zip(interesting_columns[2:], scored))
mapping['Overall'] = None
mapping

{'GeneticsScore': 'isScored_genetics',
 'OmicsScore': 'isScored_omics',
 'LiteratureScore': 'isScored_lit',
 'FlyNeuroPathScore': 'isScored_neuropath',
 'Overall': None}

In [ ]:
# Select the correct columns
interesting_ensembls = overall_scores[interesting_columns]
interesting_ensembls[interesting_ensembls.isin(['N']).any(axis=1)]
# print(overall_scores.shape)
# print(interesting_ensembls.shape)

In [132]:
def calculate_distribution(df, col, is_scored = None):

    if is_scored is not None:
        df = df[df[is_scored]=='Y']

    if df[col].dtype == object:
        df[col] = df[col].astype(float)
    
    obj = {}
    
    obj["distribution"] = list(pd.cut(df[col], bins=10, precision=3).value_counts())
    obj["min"] = np.around(df[col].min(), 4)
    obj["max"] = np.around(df[col].max(), 4)
    obj["mean"] = np.around(df[col].mean(), 4)
    discard, obj["bins"] = list(pd.cut(df[col], bins=10, precision=3, retbins=True))
    obj["bins"] = np.around(obj["bins"].tolist()[1:], 2)
    base = [0, *obj["bins"][:-1]]
    obj["bins"] = zip(base, obj["bins"])
    obj["bins"] = list(obj["bins"])
    obj["first_quartile"] = np.around(df[col].quantile(q=0.25, interpolation='midpoint'))
    obj["third_quartile"] = np.around(df[col].quantile(q=0.75, interpolation='midpoint'))
    
    return obj

In [133]:
neo_matrix = {}
for col in interesting_columns[1:]: # excludes the ENSG
    neo_matrix[col] = calculate_distribution(overall_scores, col, mapping[col])

[WARNING] <ipython-input-132-128c03b26d77>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(float)



In [134]:
print(neo_matrix)

{'Overall': {'distribution': [5141, 4253, 4143, 3590, 3146, 1733, 975, 346, 41, 18], 'min': 0.0, 'max': 7.5374, 'mean': 2.175, 'bins': [(0, 0.75), (0.75, 1.51), (1.51, 2.26), (2.26, 3.01), (3.01, 3.77), (3.77, 4.52), (4.52, 5.28), (5.28, 6.03), (6.03, 6.78), (6.78, 7.54)], 'first_quartile': 1.0, 'third_quartile': 3.0}, 'GeneticsScore': {'distribution': [4748, 4678, 4655, 3013, 2321, 1265, 567, 399, 74, 9], 'min': 0.0048, 'max': 3.0, 'mean': 1.2605, 'bins': [(0, 0.3), (0.3, 0.6), (0.6, 0.9), (0.9, 1.2), (1.2, 1.5), (1.5, 1.8), (1.8, 2.1), (2.1, 2.4), (2.4, 2.7), (2.7, 3.0)], 'first_quartile': 1.0, 'third_quartile': 2.0}, 'OmicsScore': {'distribution': [10592, 2523, 2298, 1568, 1278, 1254, 1145, 873, 852, 457], 'min': 0.0, 'max': 2.0, 'mean': 0.6768, 'bins': [(0, 0.2), (0.2, 0.4), (0.4, 0.6), (0.6, 0.8), (0.8, 1.0), (1.0, 1.2), (1.2, 1.4), (1.4, 1.6), (1.6, 1.8), (1.8, 2.0)], 'first_quartile': 0.0, 'third_quartile': 1.0}, 'LiteratureScore': {'distribution': [8008, 1390, 1103, 1086, 730, 

In [66]:
neo_matrix = {}
select_columns = overall_scores.columns[2:]
for col in select_columns:
    neo_matrix[col] = {}
    neo_matrix[col]["distribution"] = list(pd.cut(overall_scores[col], bins=10, precision=4).value_counts())
    neo_matrix[col]["min"] = np.around(overall_scores[col].min(), 4)
    neo_matrix[col]["max"] = np.around(overall_scores[col].max(), 4)
    neo_matrix[col]["mean"] = np.around(overall_scores[col].mean(), 4)
    discard, neo_matrix[col]["bins"] = list(pd.cut(overall_scores[col], bins=10, precision=4, retbins=True))
    neo_matrix[col]["bins"] = np.around(neo_matrix[col]["bins"].tolist()[1:], 4)
    base = [0, *neo_matrix[col]["bins"][:-1]]
    neo_matrix[col]["bins"] = zip(base, neo_matrix[col]["bins"])
    neo_matrix[col]["bins"] = list(neo_matrix[col]["bins"])
    neo_matrix[col]["first_quartile"] = np.around(overall_scores[col].quantile(q=0.25, interpolation='midpoint'))
    neo_matrix[col]["third_quartile"] = np.around(overall_scores[col].quantile(q=0.75, interpolation='midpoint'))

TypeError: can only concatenate str (not "float") to str

In [67]:
additional_data = [{'name': 'Overall Score', 'syn_id': 'syn25913473', 'wiki_id': '613107'},
                    {'name': 'Genetics Score', 'syn_id': 'syn25913473', 'wiki_id': '613104'},
                    {'name': 'Genomics Score', 'syn_id': 'syn25913473', 'wiki_id': '613106'},
                    {'name': 'Literature Score', 'syn_id': 'syn25913473', 'wiki_id': '613105'}
                  ]
for col, additional in zip(neo_matrix.keys(), additional_data):
    neo_matrix[col]['name'] = additional['name']
    neo_matrix[col]['syn_id'] = additional['syn_id']
    neo_matrix[col]['wiki_id'] = additional['wiki_id']
    
print(neo_matrix)


{'Overall': {'distribution': [5141, 4253, 4143, 3590, 3146, 1733, 975, 346, 41, 18], 'min': 0.0, 'max': 7.5374, 'mean': 2.175, 'bins': [(0, 0.7537), (0.7537, 1.5075), (1.5075, 2.2612), (2.2612, 3.0149), (3.0149, 3.7687), (3.7687, 4.5224), (4.5224, 5.2761), (5.2761, 6.0299), (6.0299, 6.7836), (6.7836, 7.5374)], 'first_quartile': 1.0, 'third_quartile': 3.0, 'name': 'Overall Score', 'syn_id': 'syn25913473', 'wiki_id': '613107'}, 'Overall_rank': {'distribution': [2339, 2339, 2339, 2339, 2339, 2339, 2338, 2338, 2338, 2338], 'min': 1, 'max': 23386, 'mean': 11713.4338, 'bins': [(0, 2339.5), (2339.5, 4678.0), (4678.0, 7016.5), (7016.5, 9355.0), (9355.0, 11693.5), (11693.5, 14032.0), (14032.0, 16370.5), (16370.5, 18709.0), (18709.0, 21047.5), (21047.5, 23386.0)], 'first_quartile': 5848.0, 'third_quartile': 17540.0, 'name': 'Genetics Score', 'syn_id': 'syn25913473', 'wiki_id': '613104'}, 'GeneticsScore': {'distribution': [4760, 4693, 4679, 2993, 2346, 2055, 1206, 570, 75, 9], 'min': 0.0, 'max': 

In [4]:
with open('../output/distribution_data.json', 'w') as outfile:
    json.dump(neo_matrix, outfile)